In [3]:
!pip install pypdf
!pip install -q langchain
!pip install -q torch
!pip install -q transformers
!pip install -q sentence-transformers
!pip install -q datasets
!pip install -q faiss-cpu
!pip install -U langchain-community
!pip install -U langchain transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 63.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 38.0 MB/s eta 0:00:00
   ━━━━━━━━━

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import os
import re
import json
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Path to the main dataset directory
dataset_path = '/content/drive/MyDrive/dataset'

# Initialize list to store all chunks and policies hashmap
all_chunks = []
policies = {}
folder_count = 0
pdf_count = 0

# Function to clean folder and file names
def clean_name(name):
    name = re.sub(r'\s*', '', name)
    name = re.sub(r'\.pdf$', '', name, flags=re.IGNORECASE)
    return name.strip()

# Function to get source link based on file name
def get_source_link(file_name):
    # Extract the key part
    for key in url_mapping:
        if key in file_name:  # Match partial key
            return url_mapping[key]
    return backup_source_link(folder_name)

BASE_URL = "https://www.bostonpublicschools.org/Page/5357"
def backup_source_link(folder_name):
    abbreviation = folder_name.split('(')[-1].replace(')', '')
    return f"{BASE_URL}#{abbreviation}"

# Function to process PDF
def process_pdf(pdf_path, folder_name):
    global pdf_count
    try:
        # Loading pdf
        loader = PyPDFLoader(file_path=pdf_path)
        docs_before_split = loader.load()

        if len(docs_before_split) == 0:
            print(f"Warning: No content found in {pdf_path}")
            return

        # Initialize the text splitter
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=1000,
            chunk_overlap=200,
        )

        # Split the documents into chunks
        docs_after_split = text_splitter.split_documents(docs_before_split)

        if len(docs_after_split) == 0:
            print(f"Warning: No chunks created from {pdf_path}")
            return

        # Clean folder and file names
        clean_folder_name = clean_name(folder_name)
        clean_file_name = clean_name(os.path.basename(pdf_path))

        # Add to policies hashmap
        abbreviation = folder_name.split('(')[-1].replace(')', '')
        if abbreviation not in policies:
            policies[abbreviation] = []
        policies[abbreviation].append(clean_file_name)

        # Prepare chunks with metadata
        for i, doc in enumerate(docs_after_split):
            chunk_data = {
                'folder_name': clean_folder_name,
                'file_name': clean_file_name,
                'chunk_id': i + 1,
                'content': doc.page_content,
                'source_link': get_source_link(clean_file_name),
                'backup_link': backup_source_link(clean_folder_name)
            }
            all_chunks.append(chunk_data)

        pdf_count += 1
        print(f"Processed {pdf_path}, created {len(docs_after_split)} chunks.")
    except Exception as e:
        print(f"Error processing {pdf_path}: {str(e)}")

# Walk through the dataset directory and process all PDF files
for root, dirs, files in os.walk(dataset_path):
    if files:
        folder_count += 1
    for file_name in files:
        if file_name.endswith('.pdf'):
            pdf_path = os.path.join(root, file_name)
            folder_name = os.path.basename(root)
            process_pdf(pdf_path, folder_name)

# Save chunks to a JSON file
if all_chunks:
    output_path = '/content/chunked_data_all_folders_with_links.json'
    with open(output_path, 'w') as json_file:
        json.dump(all_chunks, json_file, indent=4)
    print(f"All PDF chunks with links have been saved to {output_path}")
else:
    print("No chunks were created. Please check the input files.")

# Save policies hashmap to a JSON file
policies_output_path = '/content/policies_hashmap.json'
with open(policies_output_path, 'w') as json_file:
    json.dump(policies, json_file, indent=4)
print(f"Policies hashmap has been saved to {policies_output_path}")

# Print statistics
print(f"Number of folders traversed: {folder_count}")
print(f"Number of PDFs processed: {pdf_count}")

Processed /content/drive/MyDrive/dataset/Data and Accountability (ODA)/ODA-04 BPS Balanced Assessment System.pdf, created 11 chunks.
Processed /content/drive/MyDrive/dataset/Data and Accountability (ODA)/ODA-07 Required Documentation to Withdraw Students.pdf, created 15 chunks.
Processed /content/drive/MyDrive/dataset/Data and Accountability (ODA)/ODA-02 State Testing Security and Ethics.pdf, created 7 chunks.
Processed /content/drive/MyDrive/dataset/Data and Accountability (ODA)/ODA-03 Guidelines and Procedures for Accessing Student Data.pdf, created 19 chunks.
Processed /content/drive/MyDrive/dataset/Data and Accountability (ODA)/ODA-05 BPS Survey Administration Guidelines.pdf, created 17 chunks.
Processed /content/drive/MyDrive/dataset/Data and Accountability (ODA)/ODA-06 Participation Guidelines for Testing English Learners on Statewide Assessments.pdf, created 25 chunks.
Processed /content/drive/MyDrive/dataset/Data and Accountability (ODA)/ODA-01 Procedures for Conducting Educati

In [7]:
from langchain.document_loaders import HuggingFaceDatasetLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
from transformers import AutoTokenizer, pipeline
from langchain import HuggingFacePipeline
from langchain.chains import RetrievalQA
from langchain.schema import Document

In [8]:
# Path to the JSON file containing chunked data
json_file_path = '/content/chunked_data_all_folders_with_links.json'

# Load the JSON data
with open(json_file_path, 'r') as json_file:
    chunked_data = json.load(json_file)

# Initialize list to store documents
documents = []

# Process each entry in the JSON data
for entry in chunked_data:
    # Extract fields from JSON entry
    original_content = entry['content']
    folder_name = entry['folder_name']
    file_name = entry['file_name']
    source_link = entry.get('source_link', 'Unknown Source Link')
    backup_link = entry.get('backup_link', 'Unknown BackUp Link')

    # Create Document objects for each entry with metadata
    doc = Document(
        page_content=original_content,
        metadata={
            'folder_name': folder_name,
            'file_name': file_name,
            'source_link': source_link,
            'backup_link': backup_link
        }
    )
    documents.append(doc)

print(len(documents))

3747


In [9]:
# Define the path to the pre-trained model you want to use
modelPath = "sentence-transformers/all-MiniLM-l6-v2"

# Create a dictionary with model configuration options, specifying to use the CPU for computations
model_kwargs = {'device':'cpu'}

# Create a dictionary with encoding options, specifically setting 'normalize_embeddings' to False
encode_kwargs = {'normalize_embeddings': False}

# Initialize an instance of HuggingFaceEmbeddings with the specified parameters
embeddings = HuggingFaceEmbeddings(
    model_name=modelPath,     # Provide the pre-trained model's path
    model_kwargs=model_kwargs, # Pass the model configuration options
    encode_kwargs=encode_kwargs # Pass the encoding options
)

<ipython-input-9-a34a09b47e7d>:11: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [13]:
db = FAISS.from_documents(documents, embeddings)

In [25]:
import faiss
import json
from langchain.schema import Document
from langchain.docstore import InMemoryDocstore
from langchain.vectorstores import FAISS

# Paths for FAISS index and metadata files
faiss_index_file = "/content/faiss_index"
metadata_file = "/content/faiss_metadata.json"

# Function to save FAISS index and metadata
def save_faiss_index_and_metadata(faiss_db):
    # Save FAISS index
    faiss.write_index(faiss_db.index, faiss_index_file)
    print(f"FAISS index saved to {faiss_index_file}")

    # Serialize Document objects to JSON-serializable format
    metadata_dict = {
        key: {
            "page_content": value.page_content,
            "metadata": value.metadata
        }
        for key, value in faiss_db.docstore._dict.items()
    }

    with open(metadata_file, 'w') as f:
        json.dump(metadata_dict, f, indent=4)
    print(f"Metadata saved to {metadata_file}")

# Function to load FAISS index and metadata
def load_faiss_index_and_metadata(index_file=faiss_index_file, meta_file=metadata_file):

    try:
        loaded_index = faiss.read_index(index_file)
        print(f"FAISS index loaded from {index_file}")
    except FileNotFoundError:
        raise FileNotFoundError(f"FAISS index file not found at {index_file}")
    except Exception as e:
        raise RuntimeError(f"Failed to load FAISS index: {e}")

    try:
        with open(meta_file, 'r') as f:
            metadata_dict = json.load(f)
        print(f"Metadata loaded from {meta_file}")
    except FileNotFoundError:
        raise FileNotFoundError(f"Metadata file not found at {meta_file}")
    except json.JSONDecodeError:
        raise ValueError(f"Metadata file {meta_file} is not a valid JSON file.")

    # Reconstruct document store
    docstore = InMemoryDocstore({
        key: Document(page_content=value["page_content"], metadata=value["metadata"])
        for key, value in metadata_dict.items()
    })

    # Recreate the index_to_docstore_id mapping
    index_to_docstore_id = {i: key for i, key in enumerate(metadata_dict.keys())}

    # Recreate the FAISS database
    faiss_db = FAISS(loaded_index, docstore, index_to_docstore_id)

    print(f"FAISS database successfully recreated with {loaded_index.ntotal} vectors and {len(metadata_dict)} documents.")
    return faiss_db

save_faiss_index_and_metadata(db)

FAISS index saved to /content/faiss_index
Metadata saved to /content/faiss_metadata.json


In [17]:
# Perform a similarity search with the question
question = "What are the policies for enrollment?"
searchDocs = db.similarity_search(question)

# Loop through relevant documents and print their content, links, source links, and file names
for i, doc in enumerate(searchDocs):
    print(f"Document {i+1} Content:\n{doc.page_content}\n{'-'*80}")

    # Retrieve and print file name
    if 'file_name' in doc.metadata:
        print(f"File Name: {doc.metadata['file_name']}")
    else:
        print(f"File Name: None")

    # Retrieve and print source link
    if 'source_link' in doc.metadata:
        print(f"Source Link: {doc.metadata['source_link']}")
    else:
        print(f"Source Link: None")

    print("="*80)

Document 1 Content:
repeated annually or when a student transfers classrooms or 
schools.  
• Ensure that there is a contingency plan in place for all 
school-related venues where substitutes are utilized. 
• Encourage the students to eat all meals and snacks fully and 
on time. Be flexible with time requirements for eating and 
provide the parent or guardian with the carbohydrate 
menu. 
• Make certain that emergency communication devices (e.g., 
walkie-talkie, intercom, cell phone, etc.) are always present 
and functional. 
• Participate in the teams that develop and implement the 
student’s Section 504 Plan, other education plan, or 
individualized education program. Contribute to IEP, and 
504 implementation of diabetes related issues, where
--------------------------------------------------------------------------------
File Name: SHS-21DiabetesPolicy
Source Link: https://www.bostonpublicschools.org/Page/5357#SHS
Document 2 Content:
This policy extends to all employment and educat

In [27]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.schema import Document
import numpy as np

def remove_and_replace_document_embeddings(faiss_db, pdf_path, folder_name, file_name):
    # Identify and remove old embeddings and metadata
    old_chunk_ids = [
        key for key in faiss_db.docstore._dict.keys()
        if folder_name in key and file_name in key
    ]

    if old_chunk_ids:
        print(f"Removing {len(old_chunk_ids)} chunks for '{file_name}' in folder '{folder_name}'...")
        for chunk_id in old_chunk_ids:
            del faiss_db.docstore._dict[chunk_id]
        print(f"Old chunks removed for '{file_name}'.")

        # Remove the corresponding embeddings from FAISS index
        faiss_ids_to_remove = [int(chunk_id.split('|')[-1]) - 1 for chunk_id in old_chunk_ids]
        faiss_db.index.remove_ids(np.array(faiss_ids_to_remove, dtype=np.int64))

    # Load and split the document
    loader = PyPDFLoader(file_path=pdf_path)
    docs_before_split = loader.load()

    if not docs_before_split:
        raise ValueError(f"The PDF at {pdf_path} is empty or could not be read.")

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    docs_after_split = text_splitter.split_documents(docs_before_split)

    # Embed only the new chunks and update FAISS and metadata
    for i, chunk in enumerate(docs_after_split):
        chunk_id = f"{folder_name}|{file_name}|{i + 1}"
        page_content = getattr(chunk, 'page_content', getattr(chunk, 'text', None))

        if page_content is None:
            print(f"Chunk {i + 1} in '{file_name}' is invalid and will be skipped.")
            continue

        new_chunk = Document(
            page_content=page_content,
            metadata={
                "folder_name": folder_name,
                "file_name": file_name,
                "chunk_id": i + 1,
                "backup_link": backup_source_link(folder_name),
            }
        )
        embedding = embeddings.embed_query(page_content)

        # Add to FAISS index
        faiss_db.index.add(np.array([embedding], dtype=np.float32))

        # Add to document store
        faiss_db.docstore._dict[chunk_id] = new_chunk

    print(f"Updated document '{file_name}' with {len(docs_after_split)} chunks.")

# Assume `db` is the FAISS object already loaded
remove_and_replace_document_embeddings(
    faiss_db=db,
    pdf_path='/content/CAO-06  GPA Calculation Method.pdf',
    folder_name='Academics(CAO)',
    file_name='CAO-06GPACalculationMethod'
)

# Save the updated FAISS index and metadata
save_faiss_index_and_metadata(db)

Updated document 'CAO-06GPACalculationMethod' with 15 chunks.
FAISS index saved to /content/faiss_index
Metadata saved to /content/faiss_metadata.json


In [28]:
# Perform a similarity search with the question
question = "What are some methods to calculate GPA?"
searchDocs = db.similarity_search(question)

# Loop through relevant documents and print their content, links, source links, and file names
for i, doc in enumerate(searchDocs):
    print(f"Document {i+1} Content:\n{doc.page_content}\n{'-'*80}")

    # Retrieve and print file name
    if 'file_name' in doc.metadata:
        print(f"File Name: {doc.metadata['file_name']}")
    else:
        print(f"File Name: None")

    # Retrieve and print source link
    if 'source_link' in doc.metadata:
        print(f"Source Link: {doc.metadata['source_link']}")
    else:
        print(f"Source Link: None")

KeyError: 3748

In [5]:
# Define the URL mapping
url_mapping = {
    "ATH-01": "https://drive.google.com/file/d/1aH_x4Q7_yfSGjqbMBpPP5-M3yII4mc4o/view",
    "ATH-02": "https://drive.google.com/file/d/1VRfs42DpguRuaCKQEyGxv9Xs_hFbUpcO/view",
    "COM-01": "https://drive.google.com/file/d/1JrJBRQ87JvYvlLe0w5d-BQ0K-2eq3wjM/view",
    "COM-02": "https://drive.google.com/file/d/1Nun6F0fivaj38RKxzURNca_ptsEjNlua/view",
    "ACA-18": "https://drive.google.com/drive/folders/1XNOOmnWE4VMQ-I1stRUkAhogGw9t0I5G",
    "AMT-01": "https://drive.google.com/file/d/1sMHkNfYGn7r8VGEVCToGula73oEtYgwN/view",
    "AMT-03": "https://drive.google.com/file/d/1hEpcnEnD17bfEOGhuAgeY1G-aMUctJib/view",
    "AMT-04": "https://drive.google.com/file/d/1XJrumxGVBFNxG__pHfTKPYI6uQwuf8G5/view",
    "AMT-05": "https://drive.google.com/file/d/1xB0sAawkLC1HAhM6rbqFzh31CF_5ro03/view",
    "AMT-06": "https://drive.google.com/file/d/1fgaUp4Pn374_lBuJaW1mBZG_JUnWUGy_/view",
    "AMT-07": "https://drive.google.com/file/d/158Utqa8XW8gth30_BexWoRielUKJuke9/view",
    "CAO-01": "https://drive.google.com/drive/folders/1huzLTVFKFjveOeqp0_-AcTf-jYphCld3",
    "CAO-03": "https://drive.google.com/drive/folders/1-NCNpSa8UwYw_0MQdhmM-VV5lcBKtGEU",
    "CAO-05": "https://drive.google.com/drive/folders/1HVAuaNXNlnowmZKjhXsGRHPcaW6sf3-g",
    "CAO-06": "https://drive.google.com/drive/folders/1ThM6wTF5LH2fCiMWDrSEiJKXutrc2u5x",
    "CAO-07": "https://drive.google.com/drive/folders/1kxczLy6UXIbujEjqm6GOZtfV9laCVV0K",
    "CAO-08": "https://drive.google.com/drive/folders/1HpUfPePRLPCVkn_OP8FO9U4UtVah-U2g",
    "CAO-22": "https://drive.google.com/drive/folders/1Dbicrju9SkB4GQ8ou9_8NS9Ke240k4p2",
    "CAO-23": "https://drive.google.com/drive/folders/1e-qZveyEspiHKe3WnHlkF-_SXUMml8Gq",
    "CAO-24": "https://drive.google.com/drive/folders/1GOj3aqWOhe0WJoZU9X2mEOSEYWniowoG",
    "CAO-25": "https://drive.google.com/drive/folders/1bgSWMoxrTIqer_Uj6dArbc_yB5r07AqF",
    "CAO-27": "https://drive.google.com/drive/folders/1DdQVFUzs8FgfXjjJGBWhvmSIvEPWYVqv",
    "EL-04": "https://drive.google.com/file/d/1YWFdLk26LntE-MjUuW6pLjsg7qSDtNc8/view",
    "EL-06": "https://drive.google.com/file/d/1thfVB-Lhr6AC8l3gbx9wR_LJ-3bk7H-L/view",
    "EL-07": "https://drive.google.com/file/d/1vVZBe5fMfdRhlQeP2cOiUYoVlSFZ0Tu-/view",
    "EQT-01": "https://drive.google.com/drive/folders/1Z8sExAkw1ZYayk9OX4InngZHn7UQ-1GX",
    "EQT-02": "https://drive.google.com/drive/folders/1xqHNd5yN_rx_mStyh9Jz7Xz4keHGib44",
    "EQT-03": "https://drive.google.com/drive/folders/1Pe2hh44rr5bsveLmf0Z-_rEzeTZkl-OY",
    "EQT-04": "https://drive.google.com/drive/folders/12efP8zQjPFsMMFZAN3jgYZMUX4p2kHyh",
    "EQT-05": "https://drive.google.com/drive/folders/1KKRzKOiOSHpaPrcMOg-dI0hWfjDxz03j",
    "EQT-06": "https://drive.google.com/drive/folders/16Zj_B-UDQd0da_XNWyfnva6kkx-aBKQS",
    "EQT-07": "https://drive.google.com/drive/folders/1ITmCZNJ3OkfKmxygiCCWl1zZ3MrqzAPd",
    "EQT-08": "https://drive.google.com/drive/folders/1IC4yTlHZsLBxq-6YokiNPhJsN5ntC6ms",
    "EQT-09": "https://drive.google.com/drive/folders/1TcqkqmQmLIIFiNoz_syOmjSh-YAHkxQl",
    "EQT-10": "https://drive.google.com/drive/folders/12lvt_AyOhYHVuki08N_3kHDUVFzhp5HO",
    "FAM-01": "https://drive.google.com/file/d/1Ppltgzjz9uUMM2uW3q9dUQJDZ65D_KO4/view",
    "FAM-02": "https://drive.google.com/file/d/158OAoHtvQdrniiu40HImPIiEtDCC7rJw/view",
    "FAM-03": "https://drive.google.com/file/d/1DUa6lzjoi_65P_Iceq7zyFjOWcuEPloz/view",
    "FAM-04": "https://drive.google.com/file/d/15RonpFA4W7MI57QB3_7YxM8GMrHIc1k5/view",
    "FAM-05": "https://drive.google.com/file/d/1Yxfz3sQ9Ob-FX1MwbjHJIOR2bm4zUkaL/view",
    "FAM-06": "https://drive.google.com/file/d/1XBoUbxJns6RnOwLbVzFOvDwYrzdz2x3L/view",
    "FAM-07": "https://drive.google.com/file/d/181grRc0Qm1wunp-SbK4yZSisdWUg-MfL/view",
    "FAM-08": "https://drive.google.com/file/d/1rTUwL0SY3tBMPNoIfi5UkVSQRzbu9gyT/view",
    "FIN-01": "https://drive.google.com/file/d/1pLrKq8DDDiIigEBpZuvzPZgbQi524Irx/view",
    "FIN-02a": "https://drive.google.com/file/d/1sKflA_u87K3kYXuwE5hZt_3mXBLRyCAU/view",
    "FIN-02b": "https://drive.google.com/file/d/1aV_RsESDq2PtfaciK_CvVwQNjqn0uVv-/view",
    "FIN-03": "https://drive.google.com/file/d/1AxdMc3cgaoqR6aAhCPzzI9f_lJYmX_VI/view",
    "FIN-04": "https://drive.google.com/file/d/1jQ50oYTgMJEIcnFN_eH8AKxy7MjBFaOC/view",
    "FIN-07": "https://drive.google.com/file/d/1fOGZFif8xWwqMjBKuSxjLisD8cUMtEwh/view",
    "FIN-09": "https://drive.google.com/file/d/17_--vm5j21c3fgnkn9ZuiaZ_80_A--Lf/view",
    "FIN-10": "https://drive.google.com/file/d/1zt5hrWNvO6jiTf2K54Pmm8-j_Jntztqk/view",
    "FIN-11": "https://drive.google.com/file/d/1XJJiUpQRNOzILU-BNey-zkCvFMmllkxb/view",
    "FIN-12": "https://drive.google.com/file/d/1vL67Ryo9P6rtWbbfsBOVDpQpesht9kzZ/view",
    "FIN-14": "https://drive.google.com/file/d/1Dmgw_5-W8ifS6WUBn4SyCYJCMoF5ZM-W/view",
    "FIN-16": "https://drive.google.com/file/d/1E70jLqE_7fJjFcAjmxK5S6oX_NVfHOv4/view",
    "FIN-19": "https://drive.google.com/file/d/17GRe2UPMjttYAJgSp8n-VM4oD6-NJbq0/view",
    "FIN-20": "https://drive.google.com/file/d/1lEzhFxMwkSxgpgQuy6ZUPDT_NHK74gev/view",
    "FIN-21": "https://drive.google.com/file/d/1C77WJzv6bA_DAqmk6xlurf-9PcdArTYH/view",
    "FMT-01": "https://drive.google.com/file/d/1bQlDaxIKlS9coPOLTXsRN1N3OlzWSoCJ/view",
    "FMT-02": "https://drive.google.com/file/d/1Zwno4HydEGdSzfvkDJ7uEx4OeooMPcMF/view",
    "FMT-03": "https://drive.google.com/file/d/1738orkSnXIVEpr56NkfqQ7tMn1sOhJ4K/view",
    "FMT-04": "https://drive.google.com/file/d/14TfH360t0phHPHzTfuUugzUoDSR_DqjD/view",
    "FMT-05": "https://drive.google.com/file/d/13iXUfcrAXeQP0in09cA-YHmtf7SuRyT6/view",
    "FMT-08": "https://drive.google.com/file/d/1PyyfwMR5xMFb5rvj0TWd1ARjhZNY_biV/view",
    "FMT-09": "https://drive.google.com/file/d/1yW9E0PBVETYyd3Xu1Rf94AWhLBAI2efQ/view",
    "FMT-10": "https://drive.google.com/file/d/1lButB53moRjvYyQfuVnyG0jGBOC296sq/view",
    "FMT-11": "https://drive.google.com/file/d/1PdZzSR-hFKXfrdAGpdOwqf31lXoJwlRY/view",
    "FMT-12": "https://drive.google.com/file/d/1PCoV3DzKnwt0Of3PtZ3LrrDPrACRdxtD/view",
    "FMT-15": "https://drive.google.com/file/d/1mPINlDjRMn17VzIBbhWk7tQa464s2bzl/view",
    "FMT-18": "https://drive.google.com/file/d/1_jAqzwZzoMtS-KQEExhT5nonFda7gqbx/view",
    "FMT-19": "https://drive.google.com/file/d/1gIN6N7i6OdPIPoWRLw6t2vJbHjXtVpVy/view",
    "FMT-20": "https://drive.google.com/file/d/16ikP51nwJO58_BikMlZHxN152J2g0EUd/view",

}